# Notebook 기본 세팅

In [1]:
# Constant 선언

# 프로젝트 루트 디렉토리를 식별하기 위한 마커 파일 이름
ROOT_MARKER = "pyproject.toml"

# 한글 표시를 위한 나눔바른고딕 폰트 파일 이름
# matplotlib 의 font_manager 에 실제 폰트 파일의 위치를 넣어주어야 한다.
KOREAN_FONT_FILE = "NanumBarunGothic.ttf"

# matplotlib 에서는 font-family 의 이름으로 font 를 설정한다.
# 그래서 font 파일 그 자체가 아니라, 그 파일의 family 이름을 적어준다.
KOREAN_FONT_FAMILY = "NanumBarunGothic"

# 참고
# Font Family 와 Font File 의 차이는,
# Font Family 는 비슷한 디자인 특성을 공유하는 글꼴 그룹을 의미한다.
#
# 예를 들어 '나눔바른고딕' 폰트 패밀리는 일반(Regular), 굵게(Bold), 기울임(Italic) 등 여러 스타일을 포함할 수 있다.
# 반면, 폰트 파일(.ttf, .otf 등)은 이러한 폰트의 하나의 스타일이 저장된 실제 파일이다.
#
# 이 프로젝트에서는 폰트 용량을 줄이기 위해 일반(Regular) 인 NanumBarunGothic.ttf 만 사용한다.

In [2]:
# 프로젝트 root 를 sys.path 에 추가해서 import 구문을 사용하기 쉽게
from pathlib import Path


def find_project_root() -> Path:
    """
    pyproject.toml 파일을 기준으로 루트 디렉토리를 찾는다.
    :return: Path: 프로젝트 루트 디렉토리 경로
    """

    current_path = Path().resolve()

    while current_path != current_path.parent:
        if (current_path / ROOT_MARKER).exists():
            return current_path

        current_path = current_path.parent

    raise FileNotFoundError("프로젝트 루트 디렉토리를 찾을 수 없습니다.")


ROOT_DIR = find_project_root()

In [3]:
# matplotlib 의 한글 font 설정
import matplotlib.font_manager as fm
import matplotlib.pyplot as plt


FONTS_DATA_DIR = ROOT_DIR / "notebooks" / "fonts"


def setup_korean_font():
    font_path = FONTS_DATA_DIR / KOREAN_FONT_FILE
    fm.fontManager.addfont(font_path)

    # 폰트 설정
    plt.rcParams["font.family"] = KOREAN_FONT_FAMILY
    plt.rcParams["axes.unicode_minus"] = False


setup_korean_font()

# Naver Cloud Storage 연동해보기

In [4]:
from dotenv import load_dotenv


load_dotenv()

True

In [5]:
import os


NCLOUD_ACCESS_KEY = os.getenv("NCLOUD_ACCESS_KEY")
NCLOUD_SECRET_KEY = os.getenv("NCLOUD_SECRET_KEY")
NCLOUD_STORAGE_REGION = os.getenv("NCLOUD_STORAGE_REGION")
NCLOUD_STORAGE_BUCKET = os.getenv("NCLOUD_STORAGE_BUCKET")
NCLOUD_STORAGE_ENDPOINT_URL = os.getenv("NCLOUD_STORAGE_ENDPOINT_URL")

In [6]:
import boto3
from botocore.client import Config


s3_client = boto3.client(
    "s3",
    endpoint_url=NCLOUD_STORAGE_ENDPOINT_URL,
    aws_access_key_id=NCLOUD_ACCESS_KEY,
    aws_secret_access_key=NCLOUD_SECRET_KEY,
    region_name=NCLOUD_STORAGE_REGION,
    config=Config(signature_version="s3v4"),
)

In [7]:
from io import StringIO

import pandas as pd


DATASETS_DIR = "datasets"


def upload_dataframe_as_csv(df: pd.DataFrame, index: bool = False):
    csv_buffer = StringIO()
    df.to_csv(csv_buffer, index=index)
    return s3_client.put_object(
        Bucket=NCLOUD_STORAGE_BUCKET,
        Key=f"{DATASETS_DIR}/data.csv",
        Body=csv_buffer.getvalue(),
    )

In [8]:
example_df = pd.DataFrame({"name": ["tom", "jane", "elly"], "gender": ["m", "f", "f"], "age": [21, 23, 34]})
example_df

,name,gender,age
0,tom,m,21
1,jane,f,23
2,elly,f,34


In [9]:
upload_dataframe_as_csv(example_df)

{'ResponseMetadata': {'RequestId': '6bfd5595-9ff2-4f5d-bf32-fc76706f2abd',
  'HostId': 'ODcyNGVjZjIyOWE3OWMxM2VkN2YwYWM1NTk3YTE5YTRhYTZhOTAwZjIzYjg1OWEyNDM0NmEwMDViMDUwZjcwZQ==',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'etag': '"84a182d4f332cf1fd04213ecd3c9263e"',
   'x-amz-checksum-crc32': 'jeoaSw==',
   'x-amz-checksum-type': 'FULL_OBJECT',
   'x-amz-id-2': 'ODcyNGVjZjIyOWE3OWMxM2VkN2YwYWM1NTk3YTE5YTRhYTZhOTAwZjIzYjg1OWEyNDM0NmEwMDViMDUwZjcwZQ==',
   'x-amz-request-id': '6bfd5595-9ff2-4f5d-bf32-fc76706f2abd',
   'x-amz-version-id': 'e1f93601-3b58-11f0-879b-9cc2c468d0e9',
   'date': 'Wed, 28 May 2025 00:15:38 GMT',
   'content-length': '0',
   'server': 'Ncloud Storage'},
  'RetryAttempts': 0},
 'ETag': '"84a182d4f332cf1fd04213ecd3c9263e"',
 'ChecksumCRC32': 'jeoaSw==',
 'ChecksumType': 'FULL_OBJECT',
 'VersionId': 'e1f93601-3b58-11f0-879b-9cc2c468d0e9'}

In [10]:
def read_csv_as_dataframe(key: str) -> pd.DataFrame:
    r = s3_client.get_object(Bucket=NCLOUD_STORAGE_BUCKET, Key=key)
    file_content = r["Body"].read().decode("utf-8")
    csv_buffer = StringIO(file_content)
    return pd.read_csv(csv_buffer)

In [11]:
read_csv_as_dataframe(f"{DATASETS_DIR}/data.csv")

,name,gender,age
0,tom,m,21
1,jane,f,23
2,elly,f,34


In [12]:
s3_client.delete_object(Bucket=NCLOUD_STORAGE_BUCKET, Key=f"{DATASETS_DIR}/data.csv")

{'ResponseMetadata': {'RequestId': 'bb728e4e-9225-427d-973d-3b3aeb4e67c7',
  'HostId': 'Yzk0YmI0YmM5NGJlMjQ4N2RmNTRmYTM2OGNhNTE0NGZjNjMzNmEzYWI4NDZiMGRhMTVjNDFhYjM5NDg3YzcyZg==',
  'HTTPStatusCode': 204,
  'HTTPHeaders': {'x-amz-delete-marker': 'true',
   'x-amz-id-2': 'Yzk0YmI0YmM5NGJlMjQ4N2RmNTRmYTM2OGNhNTE0NGZjNjMzNmEzYWI4NDZiMGRhMTVjNDFhYjM5NDg3YzcyZg==',
   'x-amz-request-id': 'bb728e4e-9225-427d-973d-3b3aeb4e67c7',
   'x-amz-version-id': 'e21489fd-3b58-11f0-b92f-9cc2c468d112',
   'date': 'Wed, 28 May 2025 00:15:38 GMT',
   'server': 'Ncloud Storage'},
  'RetryAttempts': 0},
 'DeleteMarker': True,
 'VersionId': 'e21489fd-3b58-11f0-b92f-9cc2c468d112'}

# 공공 데이터 포털 - 기상청 데이터

In [13]:
WEATHER_API_KEY = os.getenv("WEATHER_API_KEY")

In [17]:
import requests


url = "http://apis.data.go.kr/1360000/AsosDalyInfoService/getWthrDataList"
params = {
    "serviceKey": WEATHER_API_KEY,
    "pageNo": "1",
    "numOfRows": "10",
    "dataType": "JSON",
    "dataCd": "ASOS",
    "dateCd": "DAY",
    "startDt": "20250101",
    "endDt": "20250131",
    "stnIds": "108",
}

res = requests.get(url, params=params)

In [18]:
response = res.json()
response

{'response': {'header': {'resultCode': '00', 'resultMsg': 'NORMAL_SERVICE'},
  'body': {'dataType': 'JSON',
   'items': {'item': [{'stnId': '108',
      'stnNm': '서울',
      'tm': '2025-01-01',
      'avgTa': '2.6',
      'minTa': '-2.5',
      'minTaHrmt': '0441',
      'maxTa': '8.9',
      'maxTaHrmt': '1540',
      'mi10MaxRn': '',
      'mi10MaxRnHrmt': '',
      'hr1MaxRn': '',
      'hr1MaxRnHrmt': '',
      'sumRnDur': '',
      'sumRn': '',
      'maxInsWs': '9.7',
      'maxInsWsWd': '290',
      'maxInsWsHrmt': '1417',
      'maxWs': '5.1',
      'maxWsWd': '250',
      'maxWsHrmt': '1519',
      'avgWs': '2.2',
      'hr24SumRws': '1861',
      'maxWd': '250',
      'avgTd': '-3.6',
      'minRhm': '49',
      'minRhmHrmt': '1543',
      'avgRhm': '64.3',
      'avgPv': '4.8',
      'avgPa': '1011.0',
      'maxPs': '1023.1',
      'maxPsHrmt': '2359',
      'minPs': '1020.0',
      'minPsHrmt': '1446',
      'avgPs': '1021.8',
      'ssDur': '9.6',
      'sumSsHr': '5.6',


In [29]:
item = response["response"]["body"]["items"]["item"][0]
item

{'stnId': '108',
 'stnNm': '서울',
 'tm': '2025-01-01',
 'avgTa': '2.6',
 'minTa': '-2.5',
 'minTaHrmt': '0441',
 'maxTa': '8.9',
 'maxTaHrmt': '1540',
 'mi10MaxRn': '',
 'mi10MaxRnHrmt': '',
 'hr1MaxRn': '',
 'hr1MaxRnHrmt': '',
 'sumRnDur': '',
 'sumRn': '',
 'maxInsWs': '9.7',
 'maxInsWsWd': '290',
 'maxInsWsHrmt': '1417',
 'maxWs': '5.1',
 'maxWsWd': '250',
 'maxWsHrmt': '1519',
 'avgWs': '2.2',
 'hr24SumRws': '1861',
 'maxWd': '250',
 'avgTd': '-3.6',
 'minRhm': '49',
 'minRhmHrmt': '1543',
 'avgRhm': '64.3',
 'avgPv': '4.8',
 'avgPa': '1011.0',
 'maxPs': '1023.1',
 'maxPsHrmt': '2359',
 'minPs': '1020.0',
 'minPsHrmt': '1446',
 'avgPs': '1021.8',
 'ssDur': '9.6',
 'sumSsHr': '5.6',
 'hr1MaxIcsrHrmt': '1200',
 'hr1MaxIcsr': '1.73',
 'sumGsr': '8.55',
 'ddMefs': '',
 'ddMefsHrmt': '',
 'ddMes': '',
 'ddMesHrmt': '',
 'sumDpthFhsc': '',
 'avgTca': '2.6',
 'avgLmac': '2.6',
 'avgTs': '-0.2',
 'minTg': '-9.5',
 'avgCm5Te': '-0.2',
 'avgCm10Te': '-0.3',
 'avgCm20Te': '0.9',
 'avgCm30Te':

In [30]:
item.keys()

dict_keys(['stnId', 'stnNm', 'tm', 'avgTa', 'minTa', 'minTaHrmt', 'maxTa', 'maxTaHrmt', 'mi10MaxRn', 'mi10MaxRnHrmt', 'hr1MaxRn', 'hr1MaxRnHrmt', 'sumRnDur', 'sumRn', 'maxInsWs', 'maxInsWsWd', 'maxInsWsHrmt', 'maxWs', 'maxWsWd', 'maxWsHrmt', 'avgWs', 'hr24SumRws', 'maxWd', 'avgTd', 'minRhm', 'minRhmHrmt', 'avgRhm', 'avgPv', 'avgPa', 'maxPs', 'maxPsHrmt', 'minPs', 'minPsHrmt', 'avgPs', 'ssDur', 'sumSsHr', 'hr1MaxIcsrHrmt', 'hr1MaxIcsr', 'sumGsr', 'ddMefs', 'ddMefsHrmt', 'ddMes', 'ddMesHrmt', 'sumDpthFhsc', 'avgTca', 'avgLmac', 'avgTs', 'minTg', 'avgCm5Te', 'avgCm10Te', 'avgCm20Te', 'avgCm30Te', 'avgM05Te', 'avgM10Te', 'avgM15Te', 'avgM30Te', 'avgM50Te', 'sumLrgEv', 'sumSmlEv', 'n99Rn', 'iscs', 'sumFogDur'])